<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Start-Creating-Network" data-toc-modified-id="Start-Creating-Network-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Start Creating Network</a></span></li><li><span><a href="#Get-edges" data-toc-modified-id="Get-edges-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Get edges</a></span></li><li><span><a href="#Get-Intersection-count" data-toc-modified-id="Get-Intersection-count-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Get Intersection count</a></span></li><li><span><a href="#Output" data-toc-modified-id="Output-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Output</a></span></li><li><span><a href="#sanity-check" data-toc-modified-id="sanity-check-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>sanity check</a></span></li></ul></div>

# Load Data

In [1]:
import networkx as nx
import pandas as pd
%matplotlib inline

In [2]:
train = pd.read_csv("../../../data/train_data.csv").drop("is_duplicate",axis=1)
#.set_index("id")
test = pd.read_csv("../../../data/test_data.csv")
#.set_index("test_id")#.drop("is_duplicate",axis=1)

train["question1"] = train.question1.str.lower()
train["question2"] = train.question2.str.lower()
test["question1"] = test.question1.str.lower()
test["question2"] = test.question2.str.lower()

In [3]:
df = pd.concat([train,test])[["question1","question2"]]
#df = train[["question1","question2"]]
df.shape

/home/jeroen/miniconda3/envs/AML/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


(404290, 2)

In [4]:
df.head(1)

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...


In [5]:
df.drop_duplicates(inplace=True)
df.shape

(403940, 2)

In [6]:
df.head()

,question1,question2
0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...
2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...
4,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?


# Start Creating Network

In [8]:
nodes_a = pd.concat([df.question1,df.question2]).unique()

In [10]:
len(nodes_a)

537168

In [11]:
nodes = pd.DataFrame(nodes_a,columns=["question"])
#nodes.index.name = "q_id"
nodes["q_id"] = nodes.index
nodes.head()

,question,q_id
0,what is the step by step guide to invest in sh...,0
1,what is the story of kohinoor (koh-i-noor) dia...,1
2,how can i increase the speed of my internet co...,2
3,why am i mentally very lonely? how can i solve...,3
4,"which one dissolve in water quikly sugar, salt...",4


In [12]:
#train["pair_id"] = train.index

In [12]:
#train.head()

,id,question1,question2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...
4,4,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?


In [14]:
edges = df.merge(nodes, how='inner', left_on="question1", right_on="question")\
        .drop("question",axis=1)\
        .rename(columns={"q_id":"q1_id"})\
        .merge(nodes, how='inner', left_on="question2", right_on="question")\
        .drop("question",axis=1)\
        .rename(columns={"q_id":"q2_id"})\
        .drop(["question1","question2"],axis=1)

In [15]:
edges.head(1)

,q1_id,q2_id
0,0,290367


In [16]:
G = nx.from_pandas_edgelist(edges,source='q1_id', target='q2_id')

In [17]:
G.number_of_nodes()

537168


# Get edges

In [18]:
d = G.degree
def get_degree(qid):
    return d[qid]

In [19]:
train.head()

,id,question1,question2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...
4,4,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?


In [20]:
def get_df_q_id(df):

    out = df.merge(nodes,left_on="question1",right_on="question",how="inner")\
    .drop(columns="question")\
    .rename(columns={"q_id":"q1_id"})\
    .merge(nodes,left_on="question2",right_on="question",how="inner")\
    .drop(columns="question")\
    .rename(columns={"q_id":"q2_id"})\
    
    out = out.drop(columns=["question1","question2"])
    
    return out

In [21]:
train.head()

,id,question1,question2
0,0,what is the step by step guide to invest in sh...,what is the step by step guide to invest in sh...
1,1,what is the story of kohinoor (koh-i-noor) dia...,what would happen if the indian government sto...
2,2,how can i increase the speed of my internet co...,how can internet speed be increased by hacking...
3,3,why am i mentally very lonely? how can i solve...,find the remainder when [math]23^{24}[/math] i...
4,4,"which one dissolve in water quikly sugar, salt...",which fish would survive in salt water?


In [22]:
train_ic = get_df_q_id(train.copy())
test_ic = get_df_q_id(test.copy())

In [23]:
def get_degrees(df):
    df['q1_degree'] = df.apply(lambda row: get_degree(row.q1_id), axis=1)
    df['q2_degree'] = df.apply(lambda row: get_degree(row.q2_id), axis=1)

    return df

In [24]:
train_ic = get_degrees(train_ic)
test_ic = get_degrees(test_ic)

In [25]:
train_ic.head(1)

,id,q1_id,q2_id,q1_degree,q2_degree
0,0,0,290367,1,1


In [26]:
print("test")
print(test_ic.shape)
print(test.shape)
print("train")
print(train_ic.shape)
print(train.shape)

test
(81126, 5)
(81126, 3)
train
(323164, 5)
(323164, 3)


# Get Intersection count 

In [27]:
def get_intersection_count(row):
    q1_id = row.q1_id
    q2_id = row.q2_id
    
    intersection_count = len(
                            set(G.neighbors(q1_id))\
                                    .intersection(set(G.neighbors(q2_id)))
                            )
    
    return(intersection_count)

In [28]:
train_ic['intersection_count'] = train_ic.apply(lambda row: get_intersection_count(row), axis=1)
test_ic['intersection_count'] = test_ic.apply(lambda row: get_intersection_count(row), axis=1)

train_ic[train_ic['intersection_count'] > 1].shape

(55898, 6)

In [29]:
print("test")
print(test_ic.shape)
print(test.shape)
print("train")
print(train_ic.shape)
print(train.shape)

test
(81126, 6)
(81126, 3)
train
(323164, 6)
(323164, 3)


In [30]:
train_ic.head(1)

,id,q1_id,q2_id,q1_degree,q2_degree,intersection_count
0,0,0,290367,1,1,0


In [31]:
test_ic.head(1)

,test_id,q1_id,q2_id,q1_degree,q2_degree,intersection_count
0,15,108580,39381,9,19,6


# Output

In [32]:
train_ic[["id","q1_degree","q2_degree","intersection_count"]]\
.to_csv("trainv2_graph_all.csv",index=False)

In [33]:
test_ic[["test_id","q1_degree","q2_degree","intersection_count"]]\
.to_csv("testv2_graph_all.csv",index=False)

# sanity check

In [46]:
train_s = pd.read_csv("train_graph.csv")
train_s2 = pd.read_csv("train_graph_all.csv")

In [49]:
sum(train_s.intersection_count > 2)

34494

In [48]:
sum(train_s2.intersection_count > 2)

44103